In [18]:
import pickle
import numpy as np
import random
from collections import defaultdict

# 读取原数据
with open('./results/func_code_to_label_30.pkl', 'rb') as f:
    data = pickle.load(f)

arr = np.array(data)
samples = arr.tolist()  # 每个元素是 [fund_code, group]

# 构建原始组 → 样本映射
group_to_samples = defaultdict(list)
for fund_code, group in samples:
    group_to_samples[group].append(fund_code)

# 开始重新分组
new_samples = []
group_size_limit = 80
new_group_id = 31  # 新组号从31开始

for old_group, fund_codes in group_to_samples.items():
    random.shuffle(fund_codes)  # ✅ 打乱
    for i in range(0, len(fund_codes), group_size_limit):
        batch = fund_codes[i:i + group_size_limit]
        for code in batch:
            new_samples.append([code, new_group_id])
        new_group_id += 1

# 保存为与原格式一致的二维数组
new_arr = np.array(new_samples)

# 可选：保存为 pkl 文件
with open('./results/func_code_to_label_random_rechunked.pkl', 'wb') as f:
    pickle.dump(new_arr.tolist(), f)

print(f"✅ 新数据保存完毕，格式与原始一致，shape: {new_arr.shape}")

✅ 新数据保存完毕，格式与原始一致，shape: (8420, 2)


In [27]:
with open('./results/func_code_to_label_30.pkl', 'rb') as f:
    data = pickle.load(f)
num = data[:, 1].max() + 1
num

30

In [25]:
import pickle
import numpy as np
import pandas as pd
from collections import Counter

arr = new_arr  # 假设 new_arr 是你刚构造好的二维数组

# 提取第二列（组号）
group_column = arr[:, 1]

# 统计每个组号出现次数
group_counts = Counter(group_column)

# 转为 DataFrame，方便查看和处理
df = pd.DataFrame(group_counts.items(), columns=['Group', 'Count']).sort_values(by='Group')

# 打印结果
print(df)

# 输出组总数
print("✅ 总共的组数:", df.shape[0])

   Group  Count
69   100     80
70   101     80
71   102     80
72   103     80
73   104     80
..   ...    ...
64    95     53
65    96     80
66    97     80
67    98     80
68    99     32

[119 rows x 2 columns]
✅ 总共的组数: 119


AttributeError: 'numpy.ndarray' object has no attribute 'items'

In [1]:
import pickle 
import pandas as pd
from sqlalchemy import create_engine, text
def insert_pred_to_sql():
    # 读取数据库连接字符串
    with open('./datasets/sql_token.pkl', 'rb') as f:
        DB_URI = pickle.load(f)

    # 创建数据库引擎
    engine = create_engine(DB_URI)

    # SQL 查询语句
    sql = text("""
        SELECT fund_code, forecast_date, pre_data, model_version, create_time, update_time
        FROM b_fund_forecast_new
        WHERE fund_code IN :codes
        ORDER BY forecast_date
    """)

    # 执行查询，传入参数（注意 tuple 中只有一个元素时加逗号）
    df = pd.read_sql_query(
        sql.bindparams(codes=tuple(['005626'])),  # 或 codes=('005626',)
        engine
    )
    
    df.to_sql('my_table', engine, if_exists='replace', index=False)
    return True

In [5]:
# import pandas as pd
# from datetime import datetime

# def convert_array_to_df(array):
#     """
#     将数组转换为包含标准日期列的 DataFrame
#     """
#     df = pd.DataFrame(array, columns=[
#         'fund_code', 'year', 'month', 'day', 'weekday',
#         'value1', 'value2', 'value3'
#     ])
#     df['year'] = df['year'].astype(int)
#     df['month'] = df['month'].astype(int)
#     df['day'] = df['day'].astype(int)
#     df['date'] = pd.to_datetime(df[['year', 'month', 'day']])
#     return df

# def find_missing_dates_from_array_list(array_list, start_date, end_date, freq='B'):
#     """
#     参数：
#     - array_list: List[np.ndarray]，每个元素是一个基金的历史数据数组
#     - start_date, end_date: 'YYYY-MM-DD'
#     - freq: 'B' 表示工作日（默认）

#     返回：
#     - dict: {fund_code: [缺失的日期列表]}
#     """
#     full_range = pd.date_range(start=start_date, end=end_date, freq=freq)
#     missing_map = {}

#     for arr in array_list:
#         df = convert_array_to_df(arr)
#         if df.empty:
#             continue

#         fund_code = df.iloc[0]['fund_code']
#         date_series = df['date']
#         missing = full_range.difference(date_series)

#         if not missing.empty:
#             missing_map[fund_code] = missing.strftime('%Y-%m-%d').tolist()

#     return missing_map

# missing_map = find_missing_dates_from_array_list(df, start_date, end_date)

# for code, dates in missing_map.items():
#     print(f"基金 {code} 缺失日期：{dates}")

In [2]:
import pandas as pd
from sqlalchemy import create_engine, text

with open('./datasets/sql_token.pkl', 'rb') as f:
    DB_URI = pickle.load(f)

# 创建数据库引擎
engine = create_engine(DB_URI)

# SQL 查询语句
sql = text("""
    SELECT fund_code, forecast_date, pre_data, model_version, create_time, update_time
    FROM b_fund_forecast_new
    WHERE fund_code IN :codes
    ORDER BY forecast_date
""")

df = pd.read_sql_query(
    sql.bindparams(codes=tuple(['005626'])),  # 或 codes=('005626',)
    engine
)
df

,fund_code,forecast_date,pre_data,model_version,create_time,update_time
0,005626,2023-07-01,"{""pre"": [1.4995611906051636, 1.526604056358337...",V3.0,2023-08-03 17:59:16,2023-09-05 14:28:30
1,005626,2023-07-02,"{""pre"": [1.4568397998809814, 1.464197158813476...",V3.0,2023-08-04 17:07:35,2023-09-05 14:28:30
2,005626,2023-07-03,"{""pre"": [1.4621272087097168, 1.466849327087402...",V3.0,2023-08-04 19:23:31,2023-09-05 14:28:30
3,005626,2023-07-04,"{""pre"": [1.4634904861450195, 1.471401810646057...",V3.0,2023-08-07 11:30:57,2023-09-05 14:28:30
4,005626,2023-07-05,"{""pre"": [1.4883460998535156, 1.518860697746276...",V3.0,2023-08-07 17:37:31,2023-09-05 14:28:30
...,...,...,...,...,...,...
468,005626,2025-04-08,"{""pre"": [1.2187557220458984, 1.235185742378234...",V3.1,2025-04-09 00:23:01,2025-04-09 00:23:01
469,005626,2025-04-09,"{""pre"": [1.234163522720337, 1.2369379997253418...",V3.1,2025-04-10 00:23:04,2025-04-10 00:23:04
470,005626,2025-04-10,"{""pre"": [1.2268850803375244, 1.230019569396972...",V3.1,2025-04-11 00:24:03,2025-04-11 00:24:03
471,005626,2025-04-12,"{""pre"": [1.2307374477386477, 1.234757900238037...",V3.1,2025-04-13 00:23:09,2025-04-13 00:23:09
